##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

# Guardar y cargar modelos

<table class="tfo-notebook-buttons" align="left">
  <td><a target="_blank" href="https://www.tensorflow.org/tutorials/keras/save_and_load"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver en TensorFlow.org</a></td>
  <td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Ejecutar en Google Colab</a></td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/es-419/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fuente en GitHub</a> </td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/es-419/tutorials/keras/save_and_load.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Descargar notebook</a> </td>
</table>

El progreso del modelo se puede guardar durante y después del entrenamiento. Esto significa que un modelo puede reanudarse donde quedó y, de este modo, evitar tiempos de entrenamiento prolongados. La posibilidad de guardar también implica que puede compartir su modelo y los demás pueden recrear su trabajo. A la hora de publicar modelos y técnicas de investigación, la mayoría de los profesionales del aprendizaje automático comparten lo siguiente:

- el código para crear el modelo, y
- los pesos entrenados, o parámetros, del modelo

Al compartir estos datos, se ayuda a los demás a comprender cómo funciona el modelo para que lo puedan probar por sí mismos con nuevos datos.

Precaución: Los modelos de TensorFlow están cifrados y es importante que tenga cuidado con los códigos que no sean confiables. Consulte [Usar TensorFlow de forma segura](https://github.com/tensorflow/tensorflow/blob/master/SECURITY.md) para obtener más información.

### Opciones

Existen distintas formas de guardar modelos de TensorFlow en función de la API que esté usando. Esta guía usa [tf.keras](https://www.tensorflow.org/guide/keras), una API de alto nivel que se usa para desarrollar y entrenar modelos en TensorFlow. Se recomienda el nuevo formato de alto nivel de `.keras` que se utiliza en este tutorial para guardar objetos de Keras, ya que ofrece un guardado sólido y eficiente basado en nombres que a menudo es más fácil de depurar que los formatos de bajo nivel o heredados. Para flujos de trabajo de guardado o serialización más avanzados, especialmente los que involucran objetos personalizados, consulte la [guía Guardar y cargar modelos de Keras](https://www.tensorflow.org/guide/keras/save_and_serialize). Por otros enfoques, consulte la [guía Usar el formato SavedModel](../../guide/saved_model.ipynb).

## Preparar

### Instalaciones e importaciones

Instale e importe TensorFlow y las dependencias:

In [ ]:
!pip install pyyaml h5py  # Required to save models in HDF5 format

In [ ]:
import os

import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

### Obtener un conjunto de datos de ejemplo

Usaremos el [conjunto de datos MNIST](http://yann.lecun.com/exdb/mnist/) para demostrar cómo guardar y cargar los pesos. Para acelerar estas ejecuciones, use los primeros 1000 ejemplos:

In [ ]:
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

### Definir un modelo

Para comenzar, genere un modelo secuencial simple:

In [ ]:
# Define a simple sequential model
def create_model():
  model = tf.keras.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10)
  ])

  model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])

  return model

# Create a basic model instance
model = create_model()

# Display the model's architecture
model.summary()

## Guardar puntos de verificación durante el entrenamiento

Puede usar un modelo entrenado sin necesidad de volver a entrenarlo o reanudar un entrenamiento donde lo dejó, en caso de que se haya interrumpido el proceso de entrenamiento. La retrollamada `tf.keras.callbacks.ModelCheckpoint` le permite guardar continuamente el modelo tanto *durante* el entrenamiento como *al final*.

### Uso de la retrollamada de punto de verificación

Cree una retrollamada  `tf.keras.callbacks.ModelCheckpoint` que guarde los pesos solo durante el entrenamiento:

In [ ]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images, test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Esta acción crea una única colección de archivos de puntos de verificación de TensorFlow que se actualiza al final de cada época:

In [ ]:
os.listdir(checkpoint_dir)

Siempre y cuando dos modelos compartan la misma arquitectura, podrá compartir los pesos entre ambos. Por lo tanto, al restaurar un modelo solo a partir de los pesos, cree un modelo con la misma arquitectura que el modelo original y, luego, configure sus pesos.

A continuación, vuelva a generar un modelo sin entrenar y evalúelo en el conjunto de prueba. Un modelo sin entrenar se ejecutará a niveles de azar (~10 % de precisión):

In [ ]:
# Create a basic model instance
model = create_model()

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100 * acc))

Luego, cargue los pesos del punto de verificación y vuelva a evaluar:

In [ ]:
# Loads the weights
model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

### Opciones de la retrollamada de punto de verificación

La retrollamada ofrece varias opciones para otorgar nombres únicos a los puntos de verificación y ajustar la frecuencia de creación de puntos de verificación.

Entrene un nuevo modelo y guarde puntos de verificación con nombres únicos cada cinco épocas:

In [ ]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

batch_size = 32

# Calculate the number of batches per epoch
import math
n_batches = len(train_images) / batch_size
n_batches = math.ceil(n_batches)    # round up the number of batches to the nearest whole integer

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    save_freq=5*n_batches)

# Create a new model instance
model = create_model()

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))

# Train the model with the new callback
model.fit(train_images, 
          train_labels,
          epochs=50, 
          batch_size=batch_size, 
          callbacks=[cp_callback],
          validation_data=(test_images, test_labels),
          verbose=0)

Ahora, revise los puntos de verificación resultantes y elija el último:

In [ ]:
os.listdir(checkpoint_dir)

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

Nota: El formato predeterminado de TensorFlow solo guarda los 5 puntos de verificación más recientes.

Para realizar la prueba, reestablezca el modelo y cargue el punto de verificación más reciente:

In [ ]:
# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## ¿Qué archivos son estos?

El código que vimos anteriormente almacena los pesos en una colección de archivos con formato de [punto de verificación](../../guide/checkpoint.ipynb) que contienen solo los pesos entrenados en formato binario. Los puntos de verificación contienen lo siguiente:

- Uno o más fragmentos que contienen los pesos de su modelo.
- Un archivo de índice que indica qué pesos se almacenaron en un qué fragmento.

Si está entrenando un modelo en una sola máquina, tendrá un fragmento con el siguiente sufijo: `.data-00000-of-00001`

## Guardar manualmente los pesos

Para guardar manualmente los pesos, use `tf.keras.Model.save_weights`. De manera predeterminada, `tf.keras` (y particularmente el método `Model.save_weights`) usa el formato de [punto de verificación](../../guide/checkpoint.ipynb) de TensorFlow con una extensión `.ckpt`. Para guardar en formato HDF5 con una extensión `.h5`, consulte la guía [Guardar y cargar modelos](https://www.tensorflow.org/guide/keras/save_and_serialize).

In [ ]:
# Save the weights
model.save_weights('./checkpoints/my_checkpoint')

# Create a new model instance
model = create_model()

# Restore the weights
model.load_weights('./checkpoints/my_checkpoint')

# Evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))

## Guardar el modelo completo

Llame `tf.keras.Model.save` para guardar la configuración de la arquitectura, los pesos y el entrenamiento de un modelo en un único archivo zip `model.keras`.

Se puede guardar un modelo completo en tres formatos de archivo diferentes (el nuevo formato `.keras` y dos formatos heredados: `SavedModel` y `HDF5`). Si guarda un modelo como `path/to/model.keras` se guarda automáticamente en el formato más reciente.

**Nota:** Para los objetos de Keras se recomienda usar el formato `.keras` de alto nivel, para guardar y recargar de forma más completa y basada en nombres, lo cual facilita la depuración. El formato SavedModel de bajo nivel y el formato H5 heredado aún son compatibles con el código existente.

Si desea cambiar al formato SavedModel, haga lo siguiente:

- Pase `save_format='tf'` a `save()`
- Pase un nombre de archivo sin extensión

Si desea cambiar al formato H5, haga lo siguiente:

- Pase `save_format='h5'` a `save()`
- Pase un nombre de archivo que termine en `.h5`

Guardar un modelo completamente funcional es muy útil, puede cargarlo en TensorFlow.js ([Saved Model](https://www.tensorflow.org/js/tutorials/conversion/import_saved_model), [HDF5](https://www.tensorflow.org/js/tutorials/conversion/import_keras)) y luego entrenarlo y ejecutarlo en navegadores web o convertirlo para ejecutarlo en dispositivos móviles mediante el uso de TensorFlow Lite ([Saved Model](https://www.tensorflow.org/lite/models/convert/#convert_a_savedmodel_recommended_), [HDF5](https://www.tensorflow.org/lite/models/convert/#convert_a_keras_model_))

*Los objetos personalizados (por ejemplo, los modelos o las capas subclasificados) requieren atención especial a la hora de guardarlos y cargarlos. Consulte la sección **Guardar objetos personalizados** que se describe más abajo.

### Nuevo formato `.keras` de alto nivel

El nuevo formato de guardado Keras v3, que se marca con la extensión `.keras`, es un formato más simple y eficiente que implementa un método de guardado basado en nombres, lo que le garantiza que lo que carga es exactamente lo que guardó, desde la perspectiva de Python. Esto simplifica mucho la tarea de depuración y constituye el formato recomendado para Keras.

La siguiente sección ilustra el método de guardado y restauración del modelo en el formato `.keras`.

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

Vuelva a cargar un modelo Keras actualizado desde el archivo zip `.keras`:

In [ ]:
new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Intente ejecutar la evaluación y predecir con el modelo cargado:

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

### Formato SavedModel

El formato SavedModel ofrece otra forma de serializar modelos. Los modelos guardados con este formato se pueden restaurar con `tf.keras.models.load_model` y son compatibles con TensorFlow Serving. La [guía SavedModel](../../guide/saved_model.ipynb) habla en detalle sobre cómo `serve/inspect` el formato SavedModel. En la siguiente sección se ilustran los pasos para guardar y restaurar el modelo.

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model as a SavedModel.
!mkdir -p saved_model
model.save('saved_model/my_model') 

El formato SavedModel es un directorio que contiene un protobuf binario y un punto de verificación de TensorFlow. Inspeccione el directorio del modelo guardado:

In [ ]:
# my_model directory
!ls saved_model

# Contains an assets folder, saved_model.pb, and variables folder.
!ls saved_model/my_model

Vuelva a cargar un modelo Keras actualizado desde el modelo guardado:

In [ ]:
new_model = tf.keras.models.load_model('saved_model/my_model')

# Check its architecture
new_model.summary()

El modelo restaurado se compila con los mismos argumentos que el modelo original. Intente ejecutar la evaluación y con el modelo cargado:

In [ ]:
# Evaluate the restored model
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

print(new_model.predict(test_images).shape)

### Formato HDF5

Keras ofrece un formato de guardado de alto nivel heredado básico que usa el estándar [HDF5](https://en.wikipedia.org/wiki/Hierarchical_Data_Format). 

In [ ]:
# Create and train a new model instance.
model = create_model()
model.fit(train_images, train_labels, epochs=5)

# Save the entire model to a HDF5 file.
# The '.h5' extension indicates that the model should be saved to HDF5.
model.save('my_model.h5') 

Ahora bien, recree el modelo a partir de ese archivo:

In [ ]:
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

# Show the model architecture
new_model.summary()

Compruebe su precisión:

In [ ]:
loss, acc = new_model.evaluate(test_images, test_labels, verbose=2)
print('Restored model, accuracy: {:5.2f}%'.format(100 * acc))

Keras guarda modelos mediante la inspección de sus arquitecturas. Esta técnica guarda todo:

- Los valores de peso
- La arquitectura del modelo
- La configuración de entrenamiento del modelo (lo que le pasa al método `.compile()`)
- El optimizador y su estado, según corresponda (esto le permite reiniciar el entrenamiento donde lo dejó)

Keras no puede guardar los optimizadores `v1.x` (desde `tf.compat.v1.train`) ya que no son compatibles con los puntos de verificación. Para los optimizadores v1.x, debe volver a compilar el modelo después de la carga; con lo que se pierde el estado del optimizador.


### Guardar objetos personalizados

Si está usando el formato SavedModel, puede omitir esta sección. La principal diferencia entre los formatos `.keras`/HDF5 de alto nivel y el formato SavedModel de bajo nivel es que los formatos `.keras`/HDF5 usan configuraciones de objetos para guardar la arquitectura del modelo, mientras que SavedModel guarda el gráfico de ejecución. Por lo tanto, los formatos SavedModel pueden guardar objetos personalizados como modelos subclasificados y capas personalizadas sin solicitar el código original. Sin embargo, como resultado, depurar SavedModels de bajo nivel puede ser una tarea más complicada, y recomendamos que use el formato `.keras` de alto nivel en su lugar, ya que está basado en nombres y es nativo de Keras.

Para guardar objetos personalizados en `.keras` y HDF5, debe hacer lo siguiente:

1. Defina un método `get_config` en su objeto y, si lo desea, un método de clase `from_config`.
    - `get_config(self)` devuelve un diccionario serializable JSON de parámetros necesarios para recrear el objeto.
    - `from_config(cls, config)` usa la configuración que devuelve `get_config` para crear un nuevo objeto. De forma predeterminada, esta función usará la configuración como kwargs de inicialización (`return cls(**config)`).
2. Pase los objetos personalizados al modelo de una de las siguientes tres maneras:
    - Registre el objeto personalizado con el decorador `@tf.keras.utils.register_keras_serializable`. **(opción recomendada)**
    - Pase el objeto directamente al argumento `custom_objects` cuando cargue el modelo. El argumento debe ser un diccionario que asigne el nombre de clase de la cadena a la clase de Python. Por ejemplo, `tf.keras.models.load_model(path, custom_objects={'CustomLayer': CustomLayer})`
    - Use `tf.keras.utils.custom_object_scope` con el objeto incluido en el argumento de diccionario `custom_objects` y coloque una llamada `tf.keras.models.load_model(path)` dentro del ámbito.

Consulte el tutorial [Escribir capas y modelos desde cero](https://www.tensorflow.org/guide/keras/custom_layers_and_models) para ver ejemplos de objetos personalizados y `get_config`.
